# Constrained Food Recommender

In this assignment, you will implement both Content Based and Collaborative Filtering Recommenders and backtracking search (or local search) on your own

100% finished homework should contain EDA, Item and User profiles generation, Content-Based Recommender, Collaborative Filtering Recommender, and soluton to CSP problem of assigning recommendations to brekfast, lunch and dinner.

## Data loading

You will work with subset of [Academic Yelp Dataset](https://www.kaggle.com/yelp-dataset/yelp-dataset) containing list of restaurants in **yelp_business.csv** and reviews of the users in **yelp_reviews.parquet**

In [106]:
import pandas as pd
import numpy as np

df_yelp_business = pd.read_csv("yelp_business.csv").drop(columns=["Unnamed: 0"])
df_yelp_reviews = pd.read_parquet("yelp_reviews.parquet")

# Leave only users with at least 3 reviews
users_count = df_yelp_reviews.groupby("user_id").count()[["business_id"]] 
users_to_use = users_count[users_count["business_id"] > 2]
df_yelp_reviews = df_yelp_reviews[df_yelp_reviews["user_id"].isin(users_to_use.index)]

## Exploratory data analysis

Here you will explore the data to find out what is the distribution of business categories, hours, places, user reviews, etc.

This step is needed to proceed later with item and user profiling and to clean your data if there are duplicates (e.g. duplicated reviews, the same businesses under different ids, categories tags which are highly correlated) or some artifacts not related to the main task.

(5 points)

In [108]:
df_yelp_reviews.head(20)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
4,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0,6TdNDKywdbjoTkizeMce8A,4,happy day finally canes near casa yes others g...,0,UgMW8bLE0QMJDCkQ1Ax5Mg
6,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0,ZayJ1zWyWgY9S_TRLT_y9Q,5,really good place simple decor amazing food gr...,1,aq_ZxGHiri48TUXJlpRkCQ
9,Ws8V970-mQt2X9CwCuT5zw,1,2009-10-13 04:16:41,0,z4BCgTkfNtCu4XY5Lp97ww,4,twice nice laid back tried weekend southern me...,3,jOERvhmK6_lo_XGUBPws_w
16,d4qwVw4PcN-_2mK2o1Ro1g,0,2015-02-02 06:28:00,0,bVTjZgRNq8ToxzvtiVrqMA,1,10pm super bowl sunday already closed weak won...,0,2hRe26HSCAWbFRn5WChK-Q
22,9Jo1pu0y2zU6ktiwQm6gNA,20,2016-12-04 03:15:21,19,sgTnHfeaEvyOoWX4TCgkuQ,4,coconut fish cafe fantastic five stars fish ca...,24,A0j21z2Q1HGic7jW6e9h7A
29,oyjkEn5fAqcGNsBbqc8LTw,0,2016-04-11 16:42:13,0,01DNCgnW8K6xSyv4krYa1w,1,good want several times hoping experience alwa...,0,XivuRnPq3D-K5cPiBTztIA
32,LkMtMHVetws5_7QfRjPtlg,4,2011-01-09 00:09:50,3,NHQWhZve7UEaGLt7IwAyDw,2,actually used one favorite hotel casinos back ...,5,H3wPBzQCKCKiHcV7GZK4kg
36,WUmosjOxt_8nWByHgm8Kkg,0,2011-01-08 23:13:25,0,-cRD6URpwks2GvP6qy8x4Q,5,knew remember eating many many many times way ...,0,6UR0kNe3C-ivlF2x1fPHNQ
42,VnOGWODJ7ZREYij0rpB-Xw,0,2013-11-14 05:08:28,0,o3kyWgk5ZMW1KlLw6-tv2g,3,brunch sunday friend meal good mean options li...,1,FmQxARdHGnkclvcVWGEAxw
54,swX3Vj3dQsz1YlCRmwNQEQ,10,2017-03-15 03:34:29,5,OoC5b4LQOmzHvzj4eI3iVg,4,great fresh seafood spot located scottsdale qu...,9,fVILhIglx6RNOXQba5t4kQ


In [109]:
df_yelp_business.describe()

,is_open,latitude,longitude,review_count,stars
count,5748.000000,5748.000000,5748.000000,5748.000000,5748.000000
mean,0.660926,40.099318,-92.418212,78.830376,3.413361
std,0.473436,4.996454,16.625489,179.539451,0.807876
min,0.000000,33.218471,-115.435431,3.000000,1.000000
25%,0.000000,35.259613,-112.072108,8.000000,3.000000
50%,1.000000,41.221624,-81.377509,24.000000,3.500000
75%,1.000000,43.708129,-79.427870,74.000000,4.000000
max,1.000000,51.288372,-72.988628,4138.000000,5.000000


In [110]:
df_yelp_reviews.describe()

,cool,funny,stars,useful
count,79934.000000,79934.000000,79934.000000,79934.000000
mean,0.819163,0.610466,3.727813,1.506756
std,1.933984,1.660643,1.171350,2.658814
min,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,3.000000,0.000000
50%,0.000000,0.000000,4.000000,1.000000
75%,1.000000,1.000000,5.000000,2.000000
max,93.000000,75.000000,5.000000,98.000000


In [111]:
# clean all the same rows in dataframe (so that there won't be equal reviews)
df_yelp_reviews = df_yelp_reviews.drop_duplicates()
df_yelp_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
4,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0,6TdNDKywdbjoTkizeMce8A,4,happy day finally canes near casa yes others g...,0,UgMW8bLE0QMJDCkQ1Ax5Mg
6,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0,ZayJ1zWyWgY9S_TRLT_y9Q,5,really good place simple decor amazing food gr...,1,aq_ZxGHiri48TUXJlpRkCQ
9,Ws8V970-mQt2X9CwCuT5zw,1,2009-10-13 04:16:41,0,z4BCgTkfNtCu4XY5Lp97ww,4,twice nice laid back tried weekend southern me...,3,jOERvhmK6_lo_XGUBPws_w
16,d4qwVw4PcN-_2mK2o1Ro1g,0,2015-02-02 06:28:00,0,bVTjZgRNq8ToxzvtiVrqMA,1,10pm super bowl sunday already closed weak won...,0,2hRe26HSCAWbFRn5WChK-Q
22,9Jo1pu0y2zU6ktiwQm6gNA,20,2016-12-04 03:15:21,19,sgTnHfeaEvyOoWX4TCgkuQ,4,coconut fish cafe fantastic five stars fish ca...,24,A0j21z2Q1HGic7jW6e9h7A


## Building recommender

First of all you should process user reviews to get the utility matrix containing ratings for users and businesses. There will be a lot of 0 in this matrix and it is better to store such matrices in the specialized data structure for sparce matrices. However, your working dataset is relatively small and we can use simple **pd.DataFrame** to proceed

In [112]:
def create_utility_matrix(reviews: pd.DataFrame, business: pd.DataFrame) -> pd.DataFrame:
    business_ids = business["business_id"].unique()
    users = reviews["user_id"].unique()
    ut_matrix = pd.DataFrame(0, columns=business_ids, index=users)
    for _, review in reviews.iterrows():
        ut_matrix.loc[review["user_id"], review["business_id"]] = review["stars"]
    # TODO: Rating normalization to (-1, 1) range (5 points)
    norm = np.linalg.norm(ut_matrix)
    normal_m = ut_matrix/norm #scaled to [0, 1]
    normalized_m = (2 * normal_m) - 1 #scaled to [-1, 1]
    return normalized_m

df_utility_matrix = create_utility_matrix(df_yelp_reviews, df_yelp_business)


In [113]:
df_utility_matrix.head()

,pQeaRpvuhoEqudo3uymHIQ,CsLQLiRoafpJPJSkNX2h5Q,lu7vtrp_bE9PnxWfA8g4Pg,vjTVxnsQEZ34XjYNS-XUpA,fnZrZlqW1Z8iWgTVDfv_MA,rVBPQdeayMYht4Uv_FOLHg,fhNf_sg-XzZ3e7HEVGuOZg,LoRef3ChgZKbxUio-sHgQg,Ga2Bt7xfqoggTypWD5VpoQ,xFc50drSPxXkcLvX5ygqrg,tLpkSwdtqqoXwU0JAGnApw,Sd75ucXKoZUM2BEfBHFUOg,lK-wuiq8b1TuU7bfbQZgsg,LAoSegVNU4wx4GTA8reB6A,-qjn24n8HYF6It9GQrQntw,ZkzutF0P_u0C0yTulwaHkA,0QjROMVW9ACKjhSEfHqNCQ,RrapAhd8ZxCj-iue7fu9FA,7j0kor_fkeYhyEpXh4OpnQ,OWkS1FXNJbozn-qPg3LWxg,j9bWpCRwpDVfwVT_V85qeA,6GHwgKNlvfIMUpFaxgBjUA,Ir_QIzs-4o9ElOtiGuxJrw,0Y5Kzo8PWHTjk0tlfAKcDQ,JcsZvx-4yovFgCXOfd6KMg,MTx-Zdl_KcU_z9G832XAjg,a6d7UcYnRvbr4t-THg4pSQ,UFU8ONTkzEkcOk61XJ8JwQ,-C0AlwLuXpcP609madJZQQ,W7hCuNdn2gzehta6eSHzgQ,8nP8ghEpT6WFcM6tfqAaGA,8k62wYhDVq1-652YbJi5eg,8Hvp1tYKiQbBgGIwkCRK5g,39lLJK_rrYY2NYomSsQdUA,kQknjbOvtPmS3NVm-RhcdQ,0y6alZmSLnPzmG5_kP5Quw,3YjPlOX3VHzKguHetiR_3g,OGVHlFHSXjHuioOvm1wVqg,Q_dh08clYUPj13GmCRzIVA,34FYKG4pHNXbM9ZRRiJaGw,...,xnVkYE3iMp_aZniiCIuD0g,DmyS9b7ykIOo7XwYt5I9wg,Xs37o78aby0o-Wmvh5yYPg,jhj-r7aH3AlJyVmGtcHi-Q,Vjzg0VOQsBWw1TA3iLol-A,HfQm-rq78QfSeJmm0i1I9Q,r0byBoB7y_IH8uicEvyCqQ,dDqG-lKO9BRadoQ9fmWP-A,6-yG0OQe-mSRoz1R72MkKg,YO0fC7aJv8PzZ1MGf_F6Vg,nvJjfEPYFXj8sJZcbt0k-Q,BGGQOJQTQerEQu0kHbT_UQ,CvonRhKDJaH155xhtpz_iw,IZUDXIq5SULhQ5RGLCdB7g,VeFqptSzekFAc3FZOpi81Q,gVLzkqIAHIWro_ZxkpjbFg,GCen6oV-_6PfMP_uKN-dZw,P3dBcZh_Hmr1wVWFZn2b-g,2Wzet4CPV0glZYXqFGyqyg,XjrjVQfKpbcvOXda-5r1jQ,u1jADJ_yMcL8bRJre7hjMQ,VBCR0KKjFfzfcvV36KnkhA,02hhtAO83rZBU1hflleElA,6pKR-h3KN7AwgGOOYBbE2A,E50mr3xobsahb77IBRwVTQ,W1iwBUfcDxoOPfhwr6EOGQ,kw3DbQo6Pqo83FSDx8HQjQ,aGMU3qMFOQzG0DT2akMfng,1SJiW_mW6IlEe7hqMTnjYg,LkMtMHVetws5_7QfRjPtlg,2SfSzEd3B7WimeZac23zhg,1dV3qNEv8nNUAX1k3qdE2w,YHCseOJ93wJh0gBcii_2qA,TJt1W9haRm2DKuoZLQ69yA,wM8QNs7uSyDqMJKjBYFPCQ,gp_bu7Ah81qaBY3M0Leffw,PUKOr5bEI87TVHjwijT1xw,zV38gkkEeJ4cVRlSWWQTfQ,H1j34TgbrVZkxeww9xlJTw,F8M0IukXQqR50IRyocRQbg
UgMW8bLE0QMJDCkQ1Ax5Mg,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
aq_ZxGHiri48TUXJlpRkCQ,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
jOERvhmK6_lo_XGUBPws_w,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2hRe26HSCAWbFRn5WChK-Q,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.992543,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
A0j21z2Q1HGic7jW6e9h7A,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,

## Content-Based Recommender

In [ ]:
def build_business_profiles(business: pd.DataFrame) -> pd.DataFrame:
    # TODO: Feature engineering (10 points)
    return pd.DataFrame(0, index=business["business_id"].unique(), columns=["item_feature"])

df_business_profiles = build_business_profiles(df_yelp_business)

In [ ]:
def build_user_profiles(utility_matrix: pd.DataFrame, 
                                   business_profiles: pd.DataFrame) -> pd.DataFrame:
    # TODO: Feature aggregation (5 points)
    return pd.DataFrame(0, index=utility_matrix.index, columns=business_profiles.columns)
df_user_profiles = build_user_profiles(df_utility_matrix, df_business_profiles)

In [ ]:
def predict_content_ratings(user_profiles: pd.DataFrame, business_profiles: pd.DataFrame) -> pd.DataFrame:
    # TODO: Distance based rating prediction (5 points)
    # TODO: Pointwise/Pairwase training based prediction (optional for 10 extra points)
    pass

df_content_predictions = predict_content_ratings(df_user_profiles, df_business_profiles)

## Collaborative Filtering Recommender

In [114]:
from sklearn.metrics.pairwise import cosine_similarity
def predict_collaborative_ratings(utility_matrix: pd.DataFrame) -> pd.DataFrame:
    # TODO: User-item collaborative a2 4filtering based rating prediction (15 points)
    # TODO: UV-decomposition based rating prediction (optional for 10 extra points)
    movie_similarity=cosine_similarity(utility_matrix)
    np.fill_diagonal(movie_similarity, 0 ) 
    return movie_similarity
    

df_collaborative_predictions = predict_collaborative_ratings(df_utility_matrix)
print(df_collaborative_predictions)

[[0.         0.99999995 0.99999993 ... 0.99999995 0.99999994 0.99999995]
 [0.99999995 0.         0.99999995 ... 0.99999997 0.99999996 0.99999997]
 [0.99999993 0.99999995 0.         ... 0.99999996 0.99999994 0.99999996]
 ...
 [0.99999995 0.99999997 0.99999996 ... 0.         0.99999996 0.99999997]
 [0.99999994 0.99999996 0.99999994 ... 0.99999996 0.         0.99999997]
 [0.99999995 0.99999997 0.99999996 ... 0.99999997 0.99999997 0.        ]]


## Evaluation

In [105]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import average_precision_score
from sklearn.metrics import coverage_error

def score_model(utility_matrix: pd.DataFrame, predicted_utility_matrix: pd.DataFrame, model_name="model_0"):
    # TODO: Implement these by hand (each metric 1 point)
    rmse_score = mean_squared_error(utility_matrix, predicted_utility_matrix)
    map_score = average_precision_score(utility_matrix, predicted_utility_matrix)
    coverage_score = coverage_error(utility_matrix, predicted_utility_matrix)
    personalization_score = 0
    intra_list_similarity_score = 0
    
    print("{} RMSE {}".format(model_name, rmse_score))
    print("{} MAP: {}".format(model_name, map_score))
    print("{} Coverage: {}".format(model_name, coverage_score))
    # print("Personalization: {}".format(model_name, personalization_score))
    # print("Intra-list similarity: {}".format(model_name, intra_list_similarity_score))    

# score_model(df_utility_matrix, df_content_predictions, "content-based approach")
score_model(df_utility_matrix, df_collaborative_predictions, "collaborative-filtering approach")

ValueError: ignored

## Constraint Satisfaction Problem

We can work with the task of planing breakfast, lunch and dinner for particular user as Constraint Satisfaction Problem with

**Domain**: {all_businesses}

**Variables**: {breakfast, lunch, dinner}

**Constraints**: {constrainst regarding individual variable, or several variables at once}

We also have predicted ratings for every business and want to have personalized plan of restaurants. So we won't only satisfy our constraints, but also would like to get the maximum cumulative rating.

Take a look on prepared constraints and finish empty constraints in similar way (some of these constraints may require analytics on business data. e.g. to finish **has_coffee_constraint** you may need to determine all the categories which may include good coffee in their menu)

In [ ]:
def is_vegetarian_constraint(business_id):
    return "vegetarian" in df_yelp_business[df_yelp_business["business_id"] == 
                                            business_id].categories.values[0].lower()

def has_coffee_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return ("coffee" in df_yelp_business[df_yelp_business["business_id"]
                                         == business_id].categories.values[0].lower() or 
            "coffee & tea" in df_yelp_business[df_yelp_business["business_id"] ==
                                         business_id].categories.values[0].lower())

def has_alcohol_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return "bars" in df_yelp_business[df_yelp_business["business_id"] ==
                                      business_id].categories.values[0].lower()

def is_open_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return df_yelp_business[df_yelp_business["business_id"] == business_id].is_open == 1

def is_open_at_date_at_time_meta_constraint(weekday, time, business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_at_monday_at_10am_constraint(business_id):
    return is_open_at_date_at_time("monday", "10:00", business_id)

def all_are_different_constraint(state):
    for time in ["breakfast", "dinner", "lunch"]:
        for _t in ["breakfast", "dinner", "lunch"]:
            if time == _t: continue
            business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[time]["business_id"]].categories.values[0].split(","))
            _business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[_t]["business_id"]].categories.values[0].split(","))
            if len(business_categories.intersection(_business_categories)) > \
                    len(business_categories.union(_business_categories)) // 2:
                return False
    return True

def all_are_in_the_same_city_constraint(state):
    # TODO: implement this constraint (1 point)
    return False

def all_are_in_the_same_region_meta_constraint(coordinates, threshold, state):
    # TODO: implement this constraint (1 point). Hint: use haversine distance https://pypi.org/project/haversine/
    return False

def all_are_in_test_region(state):
    return all_are_in_the_same_region_constraint({"lat": 40.110446, "lon": -115.301568}, 400, state)

def at_least_one_visited_place_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one familiar place
    return False

def at_least_one_has_coffee_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one place with coffee
    return False

In [ ]:
import random 

random.seed(42)
inspected_user = random.choice(df_yelp_reviews["user_id"].unique())

all_constraints = {
    "breakfast": [has_coffee_constraint, is_open_constraint, is_open_at_monday_at_10am_constraint],
    "lunch": [is_open_constraint],
    "dinner": [is_vegetarian_constraint, has_alcohol_constraint, is_open_constraint],
    "state": [at_least_one_has_coffee_constraint, at_least_one_visited_place_constraint, all_are_in_test_region,
             all_are_in_the_same_city_constraint, all_are_different_constraint]
}

def goal_test(state: dict, constraints: dict):
    cumulative_rating = state["breakfast"]["predicted_rating"]*state["lunch"]["predicted_rating"]*\
                        state["dinner"]["predicted_rating"]
    for k in constraints.keys():
        if k == "state":
            for c in constraints[k]:
                cumulative_rating *= c(state)
        else:
            for c in constraints[k]:
                cumulative_rating *= c(state[k]["business_id"])
    return cumulative_rating


def prepare_restaurants_plan(user_id: str, user_business_ratings: pd.DataFrame, constraints: dict):
    # TODO: assign breakfast, lunch and dinner by solving Constraint Satisfaction Problem 
    # maximizing total score and satisfying all the constraints (it should work with any configuration of constraints)
    
    # You can implement Backtracking (10) + Filtering (10) + Ordering (5) using goal_test
    # OR
    # Local Search (10) + Min-Conflicts heuristic (10) + Ordering (5) with modification of goal test to work as Min-Conflicts heuristic
    
    state = {"user_id" : user_id,
        "breakfast": 
                {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0},
            "lunch": 
    {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0},
            "dinner": {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0}}
    
    state_v = goal_test(state, constraints)

    
    return state

# TODO: replace df_utility_matrix with your best predictions
prepare_restaurants_plan(inspected_user, df_utility_matrix, all_constraints)